In [1]:
import torch

if torch.cuda.is_available():
    print("GPU 사용 가능!")
else:
    print("GPU 사용 불가능...")

GPU 사용 가능!


In [5]:
!unzip '/content/drive/MyDrive/archive.zip' # 압축파일 경로

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1551_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1601_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1651_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1701_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1751_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1801_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1851_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1901_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1951_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y2001_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y2251_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y2301_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import torch

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(f'Using device: {device}')


Using device: cuda


In [7]:
import os

def load_patient_data(patient_id):
    patient_dir = os.path.join("/content", str(patient_id))
    if not os.path.isdir(patient_dir):
        return None

    # 0 (양성) 및 1 (음성) 폴더 불러오기
    class_dirs = ["0", "1"]
    image_paths = []
    for class_dir in class_dirs:
        class_path = os.path.join(patient_dir, class_dir)
        if not os.path.isdir(class_path):
            continue

        for filename in os.listdir(class_path):
            image_path = os.path.join(class_path, filename)
            image_paths.append((image_path, int(class_dir)))

    return image_paths

# 8863번부터 16896번까지 환자 데이터 불러오기
patient_ids = list(range(8863, 16897))
all_image_paths = []
for patient_id in patient_ids:
    images = load_patient_data(patient_id)
    if images is not None:
        all_image_paths.extend(images)

# 데이터 셔플링
import random

random.shuffle(all_image_paths)

In [8]:
train_size = int(0.7 * len(all_image_paths))
val_size = int(0.15 * len(all_image_paths))
test_size = len(all_image_paths) - train_size - val_size

train_data = all_image_paths[:train_size]
val_data = all_image_paths[train_size:train_size + val_size]
test_data = all_image_paths[train_size + val_size:]

In [9]:
import cv2

def load_image(image_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            raise Exception("Error loading image at path: {}".format(image_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image
    except Exception as e:
        print("Error loading image:", e)
        return None

In [10]:
from torchvision import transforms, datasets

class BreastHistoDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path, label = self.image_paths[idx]
        image = load_image(image_path)
        if self.transform:
            image = self.transform(image)
        return image, label

train_dataset = BreastHistoDataset(train_data, transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
]))

val_dataset = BreastHistoDataset(val_data, transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
]))

test_dataset = BreastHistoDataset(test_data, transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
]))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [11]:
import torch
import torch.nn as nn

class UNet(nn.Module):
    def __init__(self):
        super().__init__()

        # 인코더
        self.down1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.down2 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.down3 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1)
        self.down4 = nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1)

        # 디코더
        self.up1 = nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.up2 = nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.up3 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=1, padding=1)
        self.up4 = nn.ConvTranspose2d(64, 3, kernel_size=3, stride=1, padding=1)

        # 마지막 출력
        self.out = nn.Sigmoid()

    def forward(self, x):
        # 인코더
        down1 = self.down1(x)
        down2 = self.down2(down1)
        down3 = self.down3(down2)
        down4 = self.down4(down3)

        # 디코더
        up1 = self.up1(down4)
        up2 = self.up2(up1 + down3)
        up3 = self.up3(up2 + down2)
        up4 = self.up4(up3 + down1)

        return self.out(up4)

In [22]:
from collections import Counter
from torch.utils.data import Dataset

def count_image_sizes(dataset):
    """
    데이터셋에 포함된 이미지의 크기를 확인하고 각 크기별 이미지 개수를 출력합니다.

    Args:
        dataset: 이미지 데이터셋

    Returns:
        None
    """
    assert isinstance(dataset, Dataset), "train_dataset must be a subclass of Dataset"
    sizes = Counter(tuple(image.get_size()[-2:]) for image in dataset)
    print("Image Sizes and Counts:")
    for size, count in sizes.items():
        print(f"\t{size}: {count}")


count_image_sizes(train_dataset)

AttributeError: 'tuple' object has no attribute 'get_size'

In [12]:
import torch.optim as optim

# 모델 생성
model = UNet()

# 손실 함수 및 최적화 알고리즘
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# 학습
def train(model, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target.float())
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f"Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item()}")

# 평가
def evaluate(model, val_loader, criterion):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for data, target in val_loader:
            output = model(data)
            loss = criterion(output, target.float())
            val_loss += loss.item()

    return val_loss / len(val_loader)

# 학습 및 평가 실행
for epoch in range(10):
    train(model, train_loader, optimizer, criterion, epoch)
    val_loss = evaluate(model, val_loader, criterion)
    print(f"Epoch {epoch}, Validation Loss: {val_loss}")

# 모델 저장
torch.save(model.state_dict(), "unet_breast_histopathology.pth")

RuntimeError: stack expects each tensor to be equal size, but got [3, 50, 50] at entry 0 and [3, 11, 50] at entry 28

In [ ]:
# 모델 불러오기
model = UNet()
model.load_state_dict(torch.load("unet_breast_histopathology.pth"))

# 테스트 데이터 평가
test_loss = evaluate(model, test_loader, criterion)
print(f"Test Loss: {test_loss}")

# 이미지 예측
for data, target in test_loader:
    output = model(data)
    prediction = torch.argmax(output, dim=1)

    # 이미지 출력 및 예측 결과 확인
    import matplotlib.pyplot as plt

    plt.imshow(data[0].permute(1, 2, 0))
    plt.title(f"Prediction: {prediction[0]}")
    plt.show()